# Introdução ao aprendizado por reforço

### Imports e dowloads

In [ ]:
#!pip install gym

In [ ]:
import gym
import random
import numpy as np
from collections import deque


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

### Créditos

Implementação original:
https://ieeexplore-ieee-org.ezp2.lib.umn.edu/stamp/stamp.jsp?tp=&arnumber=6313077

Baseado no jupyter-notebook:
https://github.com/gsurma/cartpole/blob/master/cartpole.py

### Descrição do problema

Um poste é preso por uma junta não acionada a um carrinho, que se move ao longo de uma pista sem atrito. O sistema é controlado aplicando uma força de +1 ou -1 ao carrinho. O pêndulo começa na vertical, e o objetivo é impedir que ele caia. Uma recompensa de +1 é fornecida para cada passo no tempo em que o poste permanece na posição vertical. O episódio termina quando o poste está a mais de 15 graus da vertical ou o carrinho se move a mais de 2,4 unidades do centro.

### Hiperparâmetros

In [ ]:
ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

### Solução

In [ ]:
class DQNSolver:
    
    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX
        
        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)
        
        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation='relu'))
        self.model.add(Dense(24, activation='relu'))
        self.model.add(Dense(self.action_space, activation='linear'))
        self.model.compile(loss='mse', optimizer=Adam(lr=LEARNING_RATE))
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))


    def act(self, state):
        """
            Toma decisões aleatórias em certos momentos,
            Prevê o q_value dado o estado atual e
            Retorna a ação.
        """
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
        
    
    def experience_replay(self):
        """
             Retreina o modelo com novas informações,
             Seleciona aleatoriamente momentos na memória.
        """
        """
             Para cada momento da memória:
             Caso o estado não tenha sido terminal, atualiza q_values. 
             Novo q_value é Recompensa + Com a ação prevista para o próximo estado * Gamma
             Se a recompensa for negativa, o q_value para esta ação no proximo estado é diminuido.
             Se a recompensa for positiva, o q_value para esta ação no proximo estado é aumentado.
             Online learning - O modelo é retreinado a cada momento lembrado.
        """
        
        if len(self.memory) < BATCH_SIZE:
            return
        
        batch = random.sample(self.memory, BATCH_SIZE)

        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            
            if not terminal:
                q_update = (reward + GAMMA + np.amax(self.model.predict(state_next)[0]))
            
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [ ]:
def cartpole():
    """
        Inicia o ambiente de treino.
        Salva tamanho do espaço observado. Neste caso é 4 (Posicao do carro, Velocidade do Carro, Angulo com vertical, 
                                    Velocidade do Topo).
        Salva o tamanho do espaco de ação. Neste caso é 2 (esquerda ou direita).
        Inicializa nosso resolvedor:
                Reinicia o ambiente a cada interacao.
                Transforma o vetor state em matrix.
                Desenha o ambiente.
                Prevê melhor proxima ação dado estado corrente.
                Realiza a ação e atualiza status.
                Se o ambiente não foi finalizado a recompensa é positiva, se não negativa.
                Transforma o vetor state em matrix.
                Faz com que a ação seja lembrada.
                Re-treina nosso modelo com a nova experiencia.       
    
    """
    env = gym.make('CartPole-v1')
    
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    
    dqn_solver = DQNSolver(observation_space, action_space)
    episode = 0
    
    while True:
        steps= 0
        episode+= 1
        
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        
        while True:
            env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            reward = reward if not terminal else -reward
            
            state_next = np.reshape(state_next, [1, observation_space])
            q_values = dqn_solver.model.predict(state)
            
            print(f'\tSTEP: {steps} - Valores Q: {q_values}\n\tAcao: {action} Recompensa: {reward} Finalizado:{terminal}')
            print(f'\n\tPROXIMO ESTADO- Pos: {state_next[0][0]} Car Vel:{state_next[0][1]} Angulo:{state_next[0][2]} Vel Top:{state_next[0][3]}')
            
            dqn_solver.remember(state, action, reward, state_next, terminal)
            dqn_solver.experience_replay()
            
            state = state_next
            steps += 1
            if terminal:
                break
        print(f'Episode: {episode} - Passos no ultimo episodio: {steps}')

In [ ]:
cartpole()